<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            پردازش زبان‌های طبیعی
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین چهارم (دسته‌بندی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        محمدمهدی ابوترابی - ۹۸۱۰۵۵۵۷
        <br><br>
        آرین احدی‌نیا - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        دسترسی به داده‌ها و مدل‌های ذخیره شده
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        به دلیل حجم بالای فایل‌های ذخیره شده، از قرار دادن آن‌ها در پوشه نوتبوک صرف نظر کرده و با اجرای قطعه کد زیر، از طریق گوگل درایو به آن‌ها دسترسی پیدا می‌کنیم.
    </p>
</div>

In [1]:
# import sys
# from google.colab import drive
# drive.mount('/content/drive')
# address = "University/Term 6/MIR/Homeworks/HW3"
# sys.path.append(f"/content/drive/My Drive/{address}")
# %cd /content/drive/My\ Drive/$address

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [ ]:
! pip install bs4
! pip install tqdm
! pip install pandas
! pip install requests
! pip install hazm
! pip install unidecode
! pip install datasets
! pip install scikit-learn
! pip install nltk
! pip install torch
! pip install transformers

In [1]:
import os
import re
import csv
import hazm
import nltk
import json
import torch
import pickle
import zipfile
import fasttext
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from datasets import Dataset
from bs4 import BeautifulSoup
from string import punctuation
from IPython.display import display
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, TextClassificationPipeline

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٦٨ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [2]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

CATEGORIES_CLASSES = {
    'Politics': 0,
    'World': 1,
    'Economy': 2,
    'Society': 3,
    'City': 4,
    'Sport': 5,
    'Science': 6,
    'Culture': 7,
    'IT': 8,
    'LifeSkills': 9,
}

CLASSES_CATEGORIES = {v: k for k, v in CATEGORIES_CLASSES.items()}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [2]:
dataset = pd.read_csv('./news_5000.csv')

In [7]:
dataset.head()

,4612103,MoL7VM5vRhfPHAszgyToRM,khabarfarsi.com/u/115997967,اجرای ویژه برنامه های گرامیداشت دهه فجر در کانون های مساجد بستان آباد,فرهنگ و هنر,http://shabestan.ir/detail/News/1141850,شبستان,Unnamed: 7,Unnamed: 8,http://media.shabestan.ir/Original/1400/11/07/IMG11431115.jpg,...,«نصیر قمصری»، در گفت وگو با خبرنگار خبرگزاری شبستان در تبریز، ضمن تبریک به مناسبت فرا رسیدن ایام الله دهه فجر و لزوم بزرگداشت این ایام با مشارکت عمومی، گفت: بیش از 15 برنامه فرهنگی، هنری، قرآنی و ورزشی به همت کانون های فرهنگی هنری مساجد بستان آباد در روزهای پرافتخار بهمن ماه اجرا می شود. سرپرست اداره فرهنگ و ارشاد اسلامی بستان آباد، افزود: برنامه های مورد نظر در کانون های فرهنگی هنری مساجد روستایی و شهری با هدف پاسداشت ارزش های انقلاب اسلامی، آگاه سازی و تجدید میثاق نسل جوان و نوجوان با آرمان های امام راحل و شهدای عظیم الشان برگزار می شود,True,True.1,False,Unnamed: 16,2022-01-29 06:38:04.225284+00:00,2022-01-29 06:45:04.225595+00:00,fa,Unnamed: 20,False.1
0,3094439,Tp4kXv3A3oB7hRV8kb5oTn,khabarfarsi.com/u/108126652,والیبال نوجوانان جهان| ستاره ای که همچنان می د...,ورزشی,https://farsnews.ir/news/14000608000920/%D9%88...,خبرگزاری فارس,NaN,"امیرمحمد گل زاده,پشت خط زن تیم والیبال نوجوانا...",https://media.farsnews.ir/Uploaded/Files/Image...,...,به گزارش خبرنگار ورزشی خبرگزاری فارس، تیم والی...,True,True,False,NaN,2021-08-30 16:33:01.639304+00:00,2021-08-30 16:45:01.639629+00:00,fa,NaN,False
1,619358,ioxLEiYLABpttJPWbuhxJv,khabarfarsi.com/u/98835151,شرکت های دانش بنیان ضربات اقتصادی همه گیری کرو...,اقتصادی,https://farsnews.ir/news/13991113000098/%D8%B4...,خبرگزاری فارس,NaN,"کرونا,دانش,علم,دانش بنیان,شرکت های دانش بنیان,...",https://media.farsnews.ir/Uploaded/Files/Image...,...,از این رو، توجه به توسعه کسب وکارهای خرد و کوچ...,True,True,False,NaN,2021-02-02 07:44:59.532751+00:00,2021-02-02 07:47:59.533036+00:00,fa,NaN,False
2,4434763,S2ktAnodxNo5rFry9WBG3Z,khabarfarsi.com/u/115005815,بزرگترین مشکل سامسونگ S21 FE 5G چیست و چرا نبا...,فناوری و ارتباطات,https://www.shahrsakhtafzar.com/fa/news/smartp...,شهر سخت افزار,NaN,NaN,https://www.shahrsakhtafzar.com/fa/images/1400...,...,اگر به یاد داشته یا به قیمت های عنوان شده در ب...,True,True,False,NaN,2022-01-09 07:36:06.211189+00:00,2022-01-09 07:45:06.211481+00:00,fa,NaN,False
3,276713,bU8DQribGWLJtWvTrV5Sb8,khabarfarsi.com/u/97546025,فیلم | بازخوانی موضع رهبر معظم انقلاب درباره ج...,سیاسی,https://www.hawzahnews.com/news/937281/%D9%81%...,حوزه,NaN,NaN,https://media.hawzahnews.com/d/2020/11/28/4/10...,...,انتشارات انقلاب اسلامی با همکاری صدای جمهوری ا...,True,True,False,NaN,2021-01-04 13:08:04.787975+00:00,2021-01-04 13:41:04.788242+00:00,fa,NaN,False
4,1401212,LSmTBshUcbwCX6CsF6hHht,khabarfarsi.com/u/101726988,هشدار : تجویز و تزریق دارو ی که ممنوعه برای بی...,علمی و پزشکی,https://www.shomanews.com/fa/tiny/news-979052,شما نیوز,NaN,NaN,https://static3.shomanews.com/thumbnail/TSnKVI...,...,مسعود مردانی روز شنبه افزود: بر اساس مصوبه کمی...,True,True,False,NaN,2021-04-13 14:56:51.396681+00:00,2021-04-13 15:00:51.396984+00:00,fa,NaN,False


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [5]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [6]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

In [19]:
def save_preprocessed_texts(texts, path="Preprocessed_texts_title.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="Preprocessed_texts_title.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)


def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()


def get_preprocessed_texts(dataset, preprocessor, mode, save=False):
    preprocessed_texts = []
    if mode == 'process':
        texts = [data_to_text(data) for _, data in dataset.iterrows()]
        preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    if mode == 'load':
        preprocessed_texts = load_preprocessed_texts()
    if save:
        save_preprocessed_texts(preprocessed_texts)
    return preprocessed_texts

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم، یا داده‌های پیش‌پردازش‌شده‌ی ذخیره‌شده را از فایل مربوطه بازیابی می‌کنیم.
    </p>
</div>

In [20]:
preprocessor = Preprocessor(stopwords_path='stopwords.txt')

In [21]:
preprocessed_texts = get_preprocessed_texts(dataset, preprocessor, mode='load', save=False)

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٣. دسته‌بندی اخبار بر اساس برچسب خبر
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این مرحله با استفاده از دو مدل متفاوت زبانی، یکی مبتنی بر روش سنتی رگرسیون لجستیک و دیگری بر پایه ترنسفرمر، شروع به یادگیری بر روی خبرهای استخراج شده می‌کنیم و در انتها، نتیجه اجرای ده کوئری را بر روی هر یک مشاهده می‌کنیم. در نهایت، ارزیابی‌های مورد بحث در داک تمرین را بر روی داده‌های تست انجام می‌دهیم.
    </p>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        کوئری‌های مورد استفاده در این بخش در زیر آمده‌اند.
    </p>
</div>

In [23]:
QUERIES = [
    "نتیجه توافق برجام",
    "حمله ارتش روسیه به اوکراین",
    "افزایش نرخ تورم در کشور",
    "آمار فوتی‌های کرونا",
    "اقدامات شهرداری تهران",
    "صعود ایران به جام جهانی",
    "بیماری آبله میمونی",
    "جشنواره فیلم فجر",
    "رونمایی از گوشی جدید اپل",
    "روش پخت غذا",
]

<div>
    <h4 style='direction:rtl;text-align:justify;'>
        ١. مدل مبتنی بر روش Logistic Regression
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این بخش از کلاس TfidfVectorizer از کتاب‌خانه‌ی sklearn استفاده کرده و مجموعه کلمات یکتای موجود در عناوین خبرها را به عنوان ورودی به آن می‌دهیم. این کلمات یکتا را از روی فایل ذخیره شده‌ی vocabulary.pickle می‌خوانیم. در ادامه، یک مدل ایجاد کرده و بردار embedding عنوان هر خبر را به روش tf-idf به دست می‌آوریم. سپس، داده‌هایمان را به دو مجموعه train و test افراز می‌کنیم. حال، با استفاده از کلاس LogisticRegression کتاب‌خانه‌ی sklearn شروع به یادگیری داده‌های train می‌کنیم.
    </p>
</div>

In [37]:
class TF_IDF:

    def __init__(self, vocabulary):
        self.vectorizer = TfidfVectorizer(max_features=2000)

    def fit_transform_vectorizer(self, dataset):
        vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        dense_vectors = vectors.todense().tolist()
        return np.array(dense_vectors)

    def save_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "wb") as file:
            pickle.dump(self.vectors.todense().tolist(), file)

    def load_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "rb") as file:
            return pickle.load(file)

In [38]:
vocabulary = {}
with open('vocabulary.pickle', "rb") as file:
    vocabulary = pickle.load(file)

In [39]:
TF_IDF_model = TF_IDF(vocabulary)
vectors = TF_IDF_model.fit_transform_vectorizer(mini_preprocessed_texts)

In [40]:
train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors, labels, test_size=0.15, shuffle=True, random_state=11)

In [43]:
classifier = LogisticRegression(max_iter=1000, random_state=0)
classifier.fit(train_vectors, train_labels)


0.86

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این قسمت، دسته‌ی هر یک از کوئری‌های مشخص شده را با استفاده از مدل یادگیری پیش‌بینی می‌کنیم.
    </p>
</div>

In [44]:
queries_class_prediction = []

for query in QUERIES:
    preprocessed_query = ' '.join(preprocessor.preprocess(query))
    query_embed = TF_IDF_model.vectorizer.transform([preprocessed_query])
    predicted_class_code = classifier.predict(query_embed)[0]
    probability = classifier.predict_proba(query_embed)[0][predicted_class_code]
    queries_class_prediction.append({
        'Query': query,
        'Predicted Category': CLASSES_CATEGORIES[predicted_class_code],
        'Probability': probability
    })

display(pd.DataFrame(queries_class_prediction))

,Query,Predicted Category,Probability
0,نتیجه توافق برجام,Politics,0.430454
1,حمله ارتش روسیه به اوکراین,World,0.799289
2,افزایش نرخ تورم در کشور,Economy,0.385289
3,آمار فوتی‌های کرونا,Society,0.480850
4,اقدامات شهرداری تهران,City,0.958167
5,صعود ایران به جام جهانی,Sport,0.483883
6,بیماری آبله میمونی,Science,0.364224
7,جشنواره فیلم فجر,Culture,0.837791
8,رونمایی از گوشی جدید اپل,IT,0.764616
9,روش پخت غذا,LifeSkills,0.498233


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این قسمت، بررسی می‌کنیم که مدل یادگیری‌مان بر روی داده‌های test چه عملکردی دارد. با اجرای قطعه کد زیر، معیار ارزیابی F1-Macro و نیز accuracy و ماتریس درهم‌ریختگی محاسبه شده و نمایش داده می‌شوند. (در این ماتریس، ستون‌ها ground truth و سطرها پیش‌بینی ما هستند.)
    </p>
</div>

In [45]:
mean_acc = classifier.score(test_vectors, test_labels)
print(f"Mean Accurracy: {round(mean_acc, 3)}\n")

predicted_labels = classifier.predict(test_vectors)
f1_macro = f1_score(test_labels, predicted_labels, labels=np.arange(10), average='macro')
print(f"F1-Macro: {round(f1_macro, 3)}\n")

confusion_mat = confusion_matrix(test_labels, predicted_labels, labels=np.arange(10))
confusion_mat_df = pd.DataFrame(confusion_mat).rename(columns=CLASSES_CATEGORIES, index=CLASSES_CATEGORIES)
display(confusion_mat_df)

Mean Accurracy: 0.86

F1-Macro: 0.857



,Politics,World,Economy,Society,City,Sport,Science,Culture,IT,LifeSkills
Politics,52,6,2,4,0,0,1,2,0,0
World,3,40,2,2,0,1,4,1,2,0
Economy,0,0,52,2,1,0,0,0,2,0
Society,4,0,0,56,4,0,0,2,1,2
City,0,0,0,4,54,0,0,0,0,0
Sport,0,0,0,0,0,62,0,0,0,0
Science,0,2,0,0,0,0,42,1,6,3
Culture,0,1,0,0,0,0,0,62,2,1
IT,0,1,3,1,0,0,1,0,61,0
LifeSkills,1,3,0,1,0,0,3,1,1,35


<div>
    <h4 style='direction:rtl;text-align:justify;'>
        ٢. مدل مبتنی بر پایه Transformer
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این بخش، با استفاده از کلاس AutoModelForSequenceClassification از کتاب‌خانه‌ی transformers نوشته‌های موجود را به دسته‌ی متناظرشان مرتبط می‌کنیم. برای این کار، از مدل ParsBERT که از پیش تمرین داده شده است به عنوان زیربنا استفاده می‌کنیم و سپس با افراز داده‌های موجود به سه بخش train و validation و test، با استفاده از دو بخش train و validation آن را بیش‌تر در راستای هدفمان که تشخیص دسته‌ی خبرهاست تمرین می‌دهیم.
        همچنین در طول تمرین دادن از یک AutoTokenizer مبتنی بر ParsBERT استفاده می‌کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        برای تمرین دادن این مدل از کلاس Trainer از همان کتاب‌خانه‌ی transformers استفاده می‌کنیم و پارامترهای تمرین دادن را هم مطابق آنچه در ادامه مشهود است با استفاده از متغیر training_args به آن می‌دهیم. در نهایت پس از پایان فرایند تمرین دادن، مدل را ذخیره می‌کنیم تا در دفعات بعد بتوانیم به صورت مستقیم آن را از فایل بازیابی کنیم و از آن استفاده کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        پس از به دست آوردن مدل، با استفاده از کلاس TextClassificationPipeline از همان کتاب‌خانه‌ی transformers پایپ‌لاین تشخیص دسته را بر اساس مدلی که به دست آورده‌ایم تعریف می‌کنیم. بعد از این کار از این پایپ‌لاین استفاده می‌کنیم تا خروجی کار را بر روی درخواست‌هایی که نوشته بودیم ببینیم و همچنین با به دست آوردن نظر مدل درباره‌ی داده‌های test و محاسبه‌ی دقت میانگین و معیار F1-Macro و مشاهده‌ی ماتریس درهم‌ریختگی، کارایی مدل را بررسی کنیم.
    </p>
</div>

In [10]:
dataset['category_code'] = dataset['category'].apply(lambda x: CATEGORIES_CLASSES[x])
labels = dataset['category_code'].to_numpy()
new_dataset = pd.DataFrame()
new_dataset['text'] = [' '.join(preprocessor.preprocess(x)) for x in dataset['title']]
new_dataset['label'] = dataset['category_code']

# Train 70%, Validation 15%, Test 15%
train_data, test_data, train_labels, test_labels = train_test_split(new_dataset, labels,  test_size=0.3, shuffle=True)
val_data, test_data, val_labels, test_labels = train_test_split(test_data, test_labels,  test_size=0.5, shuffle=True)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-zwnj-base")

tokenized_train_data = Dataset.from_pandas(train_data).map(lambda x: tokenizer(x['text'], truncation=True), batched=True)
tokenized_val_data = Dataset.from_pandas(val_data).map(lambda x: tokenizer(x['text'], truncation=True), batched=True)
tokenized_test_data = Dataset.from_pandas(test_data).map(lambda x: tokenizer(x['text'], truncation=True), batched=True)

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Parameter 'function'=<function <lambda> at 0x7fa6a7807b90> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/48 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
)

model = AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-fa-zwnj-base", num_labels=10, id2label=CLASSES_CATEGORIES)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/b

In [13]:
def train_and_save(trainer, model):
    trainer.train()
    model.save_pretrained("./new_titles_transformer_model")

# train_and_save(trainer, model)
model = AutoModelForSequenceClassification.from_pretrained("./new_titles_transformer_model")
model.eval()

loading configuration file ./new_titles_transformer_model/config.json
Model config BertConfig {
  "_name_or_path": "./new_titles_transformer_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Politics",
    "1": "World",
    "2": "Economy",
    "3": "Society",
    "4": "City",
    "5": "Sport",
    "6": "Science",
    "7": "Culture",
    "8": "IT",
    "9": "LifeSkills"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [14]:
device = torch.cuda.current_device() if torch.cuda.is_available() else -1
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=False, device=device)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این قسمت، دسته‌ی هر یک از کوئری‌های مشخص شده را با استفاده از مدل یادگیری پیش‌بینی می‌کنیم.
    </p>
</div>

In [15]:
queries_class_prediction = []

for query in QUERIES:
    preprocessed_query = ' '.join(preprocessor.preprocess(query))
    prediction = pipe(preprocessed_query)[0]
    predicted_class_code = CATEGORIES_CLASSES[prediction['label']]
    probability = prediction['score']
    queries_class_prediction.append({
        'Query': query,
        'Predicted Category': CLASSES_CATEGORIES[predicted_class_code],
        'Probability': probability
    })

display(pd.DataFrame(queries_class_prediction))

,Query,Predicted Category,Probability
0,نتیجه توافق برجام,Politics,0.994476
1,حمله ارتش روسیه به اوکراین,World,0.985831
2,افزایش نرخ تورم در کشور,Economy,0.852276
3,آمار فوتی‌های کرونا,Society,0.993497
4,اقدامات شهرداری تهران,City,0.988388
5,صعود ایران به جام جهانی,Sport,0.987941
6,بیماری آبله میمونی,Science,0.957387
7,جشنواره فیلم فجر,Culture,0.989262
8,رونمایی از گوشی جدید اپل,IT,0.832700
9,روش پخت غذا,LifeSkills,0.987767


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این قسمت، بررسی می‌کنیم که مدل یادگیری‌مان بر روی داده‌های test چه عملکردی دارد. با اجرای قطعه کد زیر، معیار ارزیابی F1-Macro و نیز accuracy و ماتریس درهم‌ریختگی محاسبه شده و نمایش داده می‌شوند. (در این ماتریس، ستون‌ها ground truth و سطرها پیش‌بینی ما هستند.)
    </p>
    <p style='direction:rtl;text-align:justify;'>
         مشاهده می‌کنیم که دسته‌هایی که مدل برای درخواست‌هایمان در بخش قبلی پیشنهاد داده همگی منطقی هستند و معیارهای F1-Macro و دقت میانگین و ماتریس درهم‌ریختگی نیز نشان‌دهنده‌ی کارکرد خوب این مدل هستند.
    </p>
</div>

In [16]:
golden_labels = test_data['label'].to_numpy()
model_test_predictions = pd.DataFrame(pipe(test_data['text'].to_list())).label.apply(lambda x: CATEGORIES_CLASSES[x]).to_numpy()

mean_acc = np.average(model_test_predictions == golden_labels)
print(f"Mean Accurracy: {round(mean_acc, 3)}\n")

f1_macro = f1_score(golden_labels, model_test_predictions, labels=np.arange(10), average='macro')
print(f"F1-Macro: {round(f1_macro, 3)}\n")

confusion_mat = confusion_matrix(golden_labels, model_test_predictions, labels=np.arange(10))
confusion_mat_df = pd.DataFrame(confusion_mat).rename(columns=CLASSES_CATEGORIES, index=CLASSES_CATEGORIES)
display(confusion_mat_df)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
Disabling tokenizer parallelism, we're using DataLoader multithreading already


Mean Accurracy: 0.871

F1-Macro: 0.832



,Politics,World,Economy,Society,City,Sport,Science,Culture,IT,LifeSkills
Politics,2026,82,85,100,25,7,5,33,2,5
World,88,288,8,6,3,3,10,16,1,4
Economy,42,6,1249,64,10,4,4,1,2,2
Society,69,8,40,1784,51,4,29,29,1,26
City,20,2,7,53,481,0,2,5,0,2
Sport,6,1,1,5,2,1225,0,14,0,1
Science,4,18,5,32,3,0,383,4,8,30
Culture,18,6,0,34,12,4,6,875,0,0
IT,1,2,5,0,0,0,14,2,37,0
LifeSkills,6,11,14,38,3,7,21,18,0,585


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        از این که وقت ارزشمند خود را برای بررسی این نوتبوک صرف کرده‌اید، صمیمانه سپاسگزاریم. (:
    </p>
</div>